Chapter 7, Question 8

In [2]:
import pandas as pd

In [3]:
from sklearn.datasets import fetch_openml

In [4]:
mnist = fetch_openml('mnist_784', version=1)

In [5]:
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [6]:
X, y = mnist["data"], mnist["target"]

In [7]:
X.shape, y.shape

((70000, 784), (70000,))

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10000, random_state=1)

In [12]:
X_train.shape, y_train.shape

((60000, 784), (60000,))

In [13]:
X_test.shape, y_test.shape

((10000, 784), (10000,))

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=10000, random_state=1)

In [15]:
X_train.shape, y_train.shape

((50000, 784), (50000,))

In [16]:
X_val.shape, y_val.shape

((10000, 784), (10000,))

In [19]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier

In [20]:
linear_svc = LinearSVC(random_state=1)
mlp_clf = MLPClassifier(random_state=1)
rf_clf = RandomForestClassifier(random_state=1)
et_clf = ExtraTreesClassifier(random_state=1)

In [21]:
classifiers = [linear_svc, mlp_clf, rf_clf, et_clf]
for classifier in classifiers:
    classifier.fit(X_train, y_train)

/Users/kalyaniasthana/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [22]:
from sklearn.metrics import accuracy_score

In [23]:
train_predictions = []
train_accuracy_scores = []
for classifier in classifiers:
    y_pred = classifier.predict(X_train)
    train_predictions.append(train_predictions)
    train_accuracy_scores.append(accuracy_score(y_pred, y_train))

In [24]:
train_accuracy_scores

[0.86836, 0.99274, 1.0, 1.0]

Oh damn, I forgot to regularize Random Forests and Extra Trees! Gotta read do everything

In [25]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=1)
et_clf = ExtraTreesClassifier(n_estimators=100, random_state=1)

In [26]:
classifiers = [linear_svc, mlp_clf, rf_clf, et_clf]
for classifier in classifiers:
    classifier.fit(X_train, y_train)

/Users/kalyaniasthana/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [27]:
train_predictions = []
train_accuracy_scores = []
for classifier in classifiers:
    y_pred = classifier.predict(X_train)
    train_predictions.append(train_predictions)
    train_accuracy_scores.append(accuracy_score(y_pred, y_train))

In [28]:
train_accuracy_scores

[0.86836, 0.99274, 1.0, 1.0]

In [29]:
val_accuracy_scores = []
for classifier in classifiers:
    val_accuracy_scores.append(accuracy_score(classifier.predict(X_val), y_val))

In [30]:
val_accuracy_scores

[0.8518, 0.9635, 0.9649, 0.969]

LinearSVC is giving shit performance and I don't want to bother to train an SVC with a kernel - takes too much time

In [31]:
from sklearn.ensemble import VotingClassifier

In [32]:
estimators = [
    ('svm', linear_svc),
    ('mlp', mlp_clf),
    ('rf', rf_clf),
    ('et', et_clf)
]

In [33]:
voting_clf = VotingClassifier(estimators=estimators, n_jobs=-1)

In [34]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('svm', LinearSVC(random_state=1)),
                             ('mlp', MLPClassifier(random_state=1)),
                             ('rf', RandomForestClassifier(random_state=1)),
                             ('et', ExtraTreesClassifier(random_state=1))],
                 n_jobs=-1)

In [35]:
voting_clf.score(X_val, y_val)

0.9679

In [37]:
[estimator.score(X_val, y_val) for estimator in voting_clf.estimators_]

[0.0, 0.0, 0.0, 0.0]

In [38]:
voting_clf.estimators

[('svm', LinearSVC(random_state=1)),
 ('mlp', MLPClassifier(random_state=1)),
 ('rf', RandomForestClassifier(random_state=1)),
 ('et', ExtraTreesClassifier(random_state=1))]

In [39]:
voting_clf.estimators_

[LinearSVC(random_state=1),
 MLPClassifier(random_state=1),
 RandomForestClassifier(random_state=1),
 ExtraTreesClassifier(random_state=1)]

In [41]:
voting_clf.set_params(svm=None)

VotingClassifier(estimators=[('svm', None),
                             ('mlp', MLPClassifier(random_state=1)),
                             ('rf', RandomForestClassifier(random_state=1)),
                             ('et', ExtraTreesClassifier(random_state=1))],
                 n_jobs=-1)

In [42]:
del voting_clf.estimators_[0]

In [43]:
voting_clf.score(X_val, y_val)

0.9703

In [44]:
voting_clf.voting = "soft"
voting_clf.score(X_val, y_val)

0.9681

In [45]:
voting_clf.voting = "hard"
voting_clf.score(X_val, y_val)

0.9703

In [46]:
voting_clf.score(X_test, y_test)

0.9713

Chapter 7, Question 9

In [47]:
import numpy as np

In [48]:
voting_clf.estimators_
# X_train_new = np.c_()

[MLPClassifier(random_state=1),
 RandomForestClassifier(random_state=1),
 ExtraTreesClassifier(random_state=1)]

In [50]:
X_train_new = np.column_stack(estimator.predict(X_val) for estimator in voting_clf.estimators_)

/Users/kalyaniasthana/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


In [51]:
X_train_new

array([[1, 1, 1],
       [0, 0, 0],
       [3, 3, 3],
       ...,
       [3, 2, 2],
       [8, 8, 8],
       [1, 1, 1]])

In [52]:
from copy import deepcopy

In [53]:
y_train_new = deepcopy(y_val)

In [54]:
X_train_new.shape, y_train_new.shape

((10000, 3), (10000,))

In [55]:
voting_clf.fit(X_train_new, y_train_new)

/Users/kalyaniasthana/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_base.py:236: FutureWarning: Using 'None' to drop an estimator from the ensemble is deprecated in 0.22 and support will be dropped in 0.24. Use the string 'drop' instead.
  "Use the string 'drop' instead.", FutureWarning


VotingClassifier(estimators=[('svm', None),
                             ('mlp', MLPClassifier(random_state=1)),
                             ('rf', RandomForestClassifier(random_state=1)),
                             ('et', ExtraTreesClassifier(random_state=1))],
                 n_jobs=-1)

In [57]:
voting_clf.score(X_train_new, y_train_new)

0.9777

In [59]:
X_test_predictions = np.empty((len(X_test), 3), dtype=np.float32)
X_test_predictions.shape
# for index, estimator in enumerate(voting_clf.estimators_):
#     X_test_predictions[:, index] = estimator.predict(X_test)

(10000, 3)

In [60]:
len(voting_clf.estimators_)

3

In [62]:
for index, estimator in enumerate([mlp_clf, rf_clf, et_clf]):
#     print(index, estimator)
    X_test_predictions[:, index] = estimator.predict(X_test)

In [64]:
y_pred = voting_clf.predict(X_test_predictions)

In [65]:
accuracy_score(y_pred, y_test)

0.9686